In [1]:
import requests
import pandas as pd
import numpy as np
from keys import *

city = "Paris"
country = "FR"
response = requests.get(f'http://api.openweathermap.org/data/2.5/forecast/?q={city},{country}&appid={OWM_key}&units=metric&lang=en')

In [2]:
data = response.json()

In [3]:
# Extract forecast list
forecast_list = data.get('list', [])

In [4]:
# Prepare lists for DataFrame columns
times = []
temperatures = []
humidities = []
weather_statuses = []
wind_speeds = []
rain_volumes = []
snow_volumes = []

for entry in forecast_list:
    times.append(entry.get('dt_txt', np.nan))
    temperatures.append(entry.get('main', {}).get('temp', np.nan))
    humidities.append(entry.get('main', {}).get('humidity', np.nan))
    weather_statuses.append(entry.get('weather', [{}])[0].get('main', np.nan))
    wind_speeds.append(entry.get('wind', {}).get('speed', np.nan))
    rain_volumes.append(entry.get('rain', {}).get('3h', np.nan))
    snow_volumes.append(entry.get('snow', {}).get('3h', np.nan))

In [5]:
# Create DataFrame
df = pd.DataFrame({
    'time': times,
    'temperature': temperatures,
    'humidity': humidities,
    'weather_status': weather_statuses,
    'wind_speed': wind_speeds,
    'rain_volume_3h': rain_volumes,
    'snow_volume_3h': snow_volumes
})

print(df.head())

                  time  temperature  humidity weather_status  wind_speed  \
0  2025-09-25 00:00:00        10.26        92         Clouds        2.30   
1  2025-09-25 03:00:00        10.22        88         Clouds        2.49   
2  2025-09-25 06:00:00         9.19        87           Rain        2.35   
3  2025-09-25 09:00:00        10.79        79           Rain        2.83   
4  2025-09-25 12:00:00        10.49        79           Rain        3.27   

   rain_volume_3h  snow_volume_3h  
0             NaN             NaN  
1             NaN             NaN  
2            0.40             NaN  
3            0.23             NaN  
4            0.19             NaN  
